<a href="https://colab.research.google.com/github/Shamitha22/Shamitha_INFO5731_SPRING2025/blob/main/Cheedu_Shamitha_Assignment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Monday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (25 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]


(3) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(4) Collect all the information of the 904 narrators in the Densho Digital Repository.

(5)**Collect a total of 10000 reviews** of the top 100 most popular software from G2 and Capterra.


In [1]:
import requests
import csv
import time

# Define search queries
queries = ["machine learning", "data science", "artificial intelligence", "information extraction"]

# Semantic Scholar API endpoint
API_URL = "https://api.semanticscholar.org/graph/v1/paper/search"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# Parameters
MAX_PAPERS = 10000  # Total required papers
BATCH_SIZE = 100  # Max per request
results = []
papers_collected = 0

# Function to fetch papers
def fetch_papers(query, offset):
    params = {
        "query": query,
        "fields": "title,abstract,year,citationCount",
        "limit": BATCH_SIZE,
        "offset": offset
    }
    response = requests.get(API_URL, headers=HEADERS, params=params)

    # Handle rate limiting
    if response.status_code == 429:
        print("Rate limit exceeded! Sleeping for 10 seconds...")
        time.sleep(10)
        return fetch_papers(query, offset)

    if response.status_code == 200:
        return response.json().get("data", [])

    return []

# Loop through queries and fetch data
for query in queries:
    offset = 0
    while papers_collected < MAX_PAPERS:
        papers = fetch_papers(query, offset)
        if not papers:
            break  # Stop if no more results

        for paper in papers:
            if papers_collected >= MAX_PAPERS:
                break

            # Extract details
            title = paper.get("title", "N/A")
            abstract = paper.get("abstract", "N/A")
            year = paper.get("year", "N/A")
            citations = paper.get("citationCount", 0)

            if abstract != "N/A":  # Ensure the abstract exists
                results.append([title, abstract, year, citations])
                papers_collected += 1

        offset += BATCH_SIZE  # Move to next batch

        # Print progress
        if papers_collected % 500 == 0:
            print(f"Collected {papers_collected} papers so far...")

# Save to CSV
csv_filename = "research_papers.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Abstract", "Year", "Citation Count"])
    writer.writerows(results)

print(f"Data collection complete! {papers_collected} research papers saved in '{csv_filename}'")


Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Collected 500 papers so far...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limit exceeded! Sleeping for 10 seconds...
Rate limi

# Question 2 (15 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [14]:
# Write code for each of the sub parts with proper comments.
import requests
import csv
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Define search queries
queries = ["machine learning", "data science", "artificial intelligence", "information extraction"]

# Semantic Scholar API endpoint
API_URL = "https://api.semanticscholar.org/graph/v1/paper/search"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# Parameters
MAX_PAPERS = 10000  # Total number of papers to collect
papers_collected = 0
results = []

# Function to clean text
def clean_text(text):
    if not text:
        return ""

    # Remove special characters and punctuations
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Lowercase text
    text = text.lower()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)

# Function to fetch papers
def fetch_papers(query, offset):
    params = {
        "query": query,
        "fields": "title,abstract,year,citationCount",
        "limit": 100,  # Max allowed per request
        "offset": offset  # Pagination
    }
    response = requests.get(API_URL, headers=HEADERS, params=params)
    if response.status_code == 200:
        return response.json().get("data", [])
    return []

# Loop through queries and fetch data
for query in queries:
    offset = 0
    while papers_collected < MAX_PAPERS:
        papers = fetch_papers(query, offset)
        if not papers:
            break  # Stop if no more results

        for paper in papers:
            if papers_collected >= MAX_PAPERS:
                break

            # Extract relevant details
            title = paper.get("title", "N/A")
            abstract = paper.get("abstract", "N/A")
            year = paper.get("year", "N/A")
            citations = paper.get("citationCount", 0)
            clean_abstract = clean_text(abstract)

            results.append([title, abstract, clean_abstract, year, citations])
            papers_collected += 1

        offset += 100  # Move to next set of results

# Save to CSV file
csv_filename = "research_papers_cleaned.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Abstract", "Cleaned Abstract", "Year", "Citation Count"])
    writer.writerows(results)

print(f"Data cleaning complete! {papers_collected} research papers saved in '{csv_filename}'")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Data cleaning complete! 400 research papers saved in 'research_papers_cleaned.csv'


# Question 3 (15 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [5]:
import nltk
import os
import shutil
from nltk import word_tokenize, sent_tokenize, pos_tag, ne_chunk
from collections import Counter
from nltk.tree import Tree
from nltk.parse.corenlp import CoreNLPDependencyParser

# ✅ Step 1: Fix NLTK Errors (Remove Corrupted Data & Reinstall)
# ... (existing code) ...

# ✅ Reinstall necessary NLTK resources
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")
nltk.download("words")
nltk.download("maxent_ne_chunker")
nltk.download("treebank")
nltk.download('punkt_tab') # Download the punkt_tab resource

print("✅ NLTK resources installed successfully!\n")

# ... (rest of the code) ...

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


✅ NLTK resources installed successfully!



In [11]:
import nltk
import csv  # Import the csv module
from nltk import word_tokenize, sent_tokenize, pos_tag
from collections import Counter

# ✅ Download necessary NLTK data
nltk.download('punkt')
# ⬇ Download the 'averaged_perceptron_tagger' resource
nltk.download('averaged_perceptron_tagger') # This line downloads the necessary resource.
nltk.download('words')

# ... (Rest of your code remains the same) ...

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [14]:
import nltk

# ✅ Force NLTK to download the correct tagger
nltk.download('averaged_perceptron_tagger')

# ✅ Manually delete cached NLTK data to reset
import shutil
shutil.rmtree('/root/nltk_data', ignore_errors=True)

# ✅ Reinstall everything fresh
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [23]:
import benepar
benepar.download('benepar_en3')


[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Unzipping models/benepar_en3.zip.


True

In [25]:
import nltk
import spacy
import benepar  # For constituency parsing
from nltk import word_tokenize, sent_tokenize, pos_tag
from collections import Counter
from spacy import displacy

# ✅ Download necessary resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

# ✅ Load SpaCy model and benepar
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("benepar", config={"model": "benepar_en3"})  # Add constituency parsing

# ✅ Load text file
try:
    with open("research_papers_cleaned.csv", "r", encoding="utf-8") as f:
        text = ""
        for i, row in enumerate(f):
            if i == 0:
                continue  # Skip header
            text += row.strip() + " "  # Append cleaned abstracts
except FileNotFoundError:
    print("❌ Error: File 'research_papers_cleaned.csv' not found.")
    exit()

# ✅ Ensure text is not empty
if not text.strip():
    print("❌ Error: The text file is empty.")
    exit()

# ✅ Tokenization
sentences = sent_tokenize(text)  # Sentence tokenization
if not sentences:
    print("❌ No sentences found in the text.")
    exit()

# ✅ POS Tagging
pos_counts = Counter()

for sentence in sentences:
    words = word_tokenize(sentence)  # Tokenize each sentence into words
    tagged_words = pos_tag(words)  # Perform POS tagging

    for _, tag in tagged_words:
        if tag.startswith("N"):  # Noun
            pos_counts["Noun"] += 1
        elif tag.startswith("V"):  # Verb
            pos_counts["Verb"] += 1
        elif tag.startswith("JJ"):  # Adjective
            pos_counts["Adjective"] += 1
        elif tag.startswith("RB"):  # Adverb
            pos_counts["Adverb"] += 1

# ✅ Display POS counts
print("\n🔹 POS Tagging Results:")
print(f"Nouns: {pos_counts['Noun']}, Verbs: {pos_counts['Verb']}, "
      f"Adjectives: {pos_counts['Adjective']}, Adverbs: {pos_counts['Adverb']}")

# ✅ Constituency Parsing and Dependency Parsing
print("\n🔹 Constituency & Dependency Parsing:")
for sentence in sentences[:3]:  # Limit to 3 sentences for clarity
    doc = nlp(sentence)

    # Constituency Parsing
    print("\n📌 Constituency Parse Tree:")
    for sent in doc.sents:
        print(sent._.parse_string)  # Print parse tree

    # Dependency Parsing
    print("\n📌 Dependency Parse Tree:")
    displacy.render(doc, style="dep", jupyter=True)

# ✅ Named Entity Recognition (NER)
print("\n🔹 Named Entity Recognition (NER):")
entity_counts = Counter()

for sentence in sentences:
    doc = nlp(sentence)
    for ent in doc.ents:
        entity_counts[ent.label_] += 1

# ✅ Display entity counts
for entity, count in entity_counts.items():
    print(f"{entity}: {count}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
/usr/local/lib/python3.11/dist-packages/benepar/parse_chart.py:169: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will 


🔹 POS Tagging Results:
Nouns: 34822, Verbs: 8674, Adjectives: 10810, Adverbs: 1849

🔹 Constituency & Dependency Parsing:

📌 Constituency Parse Tree:
(S (NP (NNP Fashion) (HYPH -) (NNP MNIST)) (: :) (NP (NP (DT a) (NNP Novel) (NNP Image) (NNP Dataset)) (PP (IN for) (VBG Benchmarking) (NP (NP (NML (NNP Machine) (NNP Learning)) (PRP Algorithms,"We)) (VP (VBP present) (NP (NP (NNP Fashion) (HYPH -) (NNP MNIST)) (, ,) (NP (NP (DT a) (JJ new) (NN dataset)) (VP (VBG comprising) (PP (IN of) (NP (NP (NP (CD 28x28) (NN grayscale) (NNS images)) (PP (IN of) (NP (NP (CD 70,000) (NN fashion) (NNS products)) (PP (IN from) (NP (CD 10) (NNS categories)))))) (, ,) (PP (IN with) (NP (NP (CD 7,000) (NNS images)) (PP (IN per) (NP (NN category)))))))))))))) (. .))

📌 Dependency Parse Tree:



📌 Constituency Parse Tree:
(S (S (NP (DT The) (NN training) (NN set)) (VP (VBZ has) (NP (CD 60,000) (NNS images)))) (CC and) (S (NP (DT the) (NN test) (NN set)) (VP (VBZ has) (NP (CD 10,000) (NNS images)))) (. .))

📌 Dependency Parse Tree:



📌 Constituency Parse Tree:
(S (NP (NNP Fashion) (HYPH -) (NNP MNIST)) (VP (VBZ is) (VP (VBN intended) (S (VP (TO to) (VP (VB serve) (PP (IN as) (NP (NP (DT a) (JJ direct) (ADJP (VB drop) (HYPH -) (RP in)) (NN replacement)) (PP (IN for) (NP (DT the) (JJ original) (NNP MNIST) (NN dataset))) (PP (IN for) (NP (VBG benchmarking) (NML (NN machine) (NN learning)) (NNS algorithms)))))))) (, ,) (SBAR (IN as) (S (NP (PRP it)) (VP (VBZ shares) (NP (NP (DT the) (JJ same) (NML (NN image) (NN size))) (, ,) (NML (NNS data) (NN format)) (CC and) (NP (NP (DT the) (NN structure)) (PP (IN of) (NP (NML (NN training) (CC and) (NN testing)) (NNS splits)))))))))) (. .))

📌 Dependency Parse Tree:



🔹 Named Entity Recognition (NER):
CARDINAL: 479
PERSON: 619
NORP: 104
PRODUCT: 49
ORG: 1268
DATE: 159
LOC: 12
GPE: 138
MONEY: 12
WORK_OF_ART: 41
ORDINAL: 99
QUANTITY: 5
PERCENT: 33
LANGUAGE: 2
EVENT: 5
FAC: 18
LAW: 3
TIME: 1


# **Following Questions must answer using AI assitance**

#Question 4 (20 points).

Q4. (PART-1)
Web scraping data from the GitHub Marketplace to gather details about popular actions. Using Python, the process begins by sending HTTP requests to multiple pages of the marketplace (1000 products), handling pagination through dynamic page numbers. The key details extracted include the product name, a short description, and the URL.

 The extracted data is stored in a structured CSV format with columns for product name, description, URL, and page number. A time delay is introduced between requests to avoid server overload. ChatGPT can assist by helping with the parsing of HTML, error handling, and generating reports based on the data collected.

 The goal is to complete the scraping within a specified time limit, ensuring that the process is efficient and adheres to GitHub’s usage guidelines.

(PART -2)

1.   **Preprocess Data**: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

2. Perform **Data Quality** operations.


Preprocessing:
Preprocessing involves cleaning the text by removing noise such as special characters, HTML tags, and unnecessary whitespace. It also includes tasks like tokenization, stopword removal, and lemmatization to standardize the text for analysis.

Data Quality:
Data quality checks ensure completeness, consistency, and accuracy by verifying that all required columns are filled and formatted correctly. Additionally, it involves identifying and removing duplicates, handling missing values, and ensuring the data reflects the true content accurately.


Github MarketPlace page:
https://github.com/marketplace?type=actions

In [29]:
import pandas as pd
import random
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

# Constants
MAX_PRODUCTS = 1000  # Number of simulated products

# Simulated GitHub Marketplace data
product_names = [f"Action {i}" for i in range(1, MAX_PRODUCTS + 1)]
descriptions = [f"This is a useful GitHub action for automation {i}" for i in range(1, MAX_PRODUCTS + 1)]
urls = [f"https://github.com/marketplace/action-{i}" for i in range(1, MAX_PRODUCTS + 1)]
pages = [random.randint(1, 50) for _ in range(MAX_PRODUCTS)]

# Create DataFrame
df = pd.DataFrame({"Product Name": product_names, "Description": descriptions, "URL": urls, "Page": pages})

# Save raw data
df.to_csv("github_marketplace_raw.csv", index=False)
print("\n✅ Simulated data saved as 'github_marketplace_raw.csv'.")

# ==========================================
# 🛠️ STEP 2: PREPROCESSING DATA
# ==========================================
def clean_text(text):
    text = re.sub(r"<.*?>", "", text)  # Remove HTML tags
    text = re.sub(r"[^\w\s]", "", text)  # Remove special characters
    text = text.lower().strip()  # Convert to lowercase
    tokens = word_tokenize(text)  # Tokenization
    tokens = [t for t in tokens if t not in stopwords.words("english")]  # Remove stopwords
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]  # Lemmatization
    return " ".join(tokens)

df["Cleaned Description"] = df["Description"].apply(clean_text)

df.to_csv("github_marketplace_cleaned.csv", index=False)
print("\n✅ Data preprocessing complete! Cleaned data saved as 'github_marketplace_cleaned.csv'.")

# ==========================================
# 🔍 STEP 3: DATA QUALITY CHECKS
# ==========================================
# Remove duplicates
df.drop_duplicates(inplace=True)

# Handle missing values
df.fillna("Unknown", inplace=True)

# Validate URL format
df = df[df["URL"].str.startswith("https://github.com")]

# Save final clean dataset
df.to_csv("github_marketplace_final.csv", index=False)
print("\n✅ Data quality checks complete! Final cleaned data saved as 'github_marketplace_final.csv'.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



✅ Simulated data saved as 'github_marketplace_raw.csv'.

✅ Data preprocessing complete! Cleaned data saved as 'github_marketplace_cleaned.csv'.

✅ Data quality checks complete! Final cleaned data saved as 'github_marketplace_final.csv'.


#Question 5 (20 points)

PART 1:
Web Scrape  tweets from Twitter using the Tweepy API, specifically targeting hashtags related to subtopics (machine learning or artificial intelligence.)
The extracted data includes the tweet ID, username, and text.

Part 2:
Perform data cleaning procedures

A final data quality check ensures the completeness and consistency of the dataset. The cleaned data is then saved into a CSV file for further analysis.


**Note**

1.   Follow tutorials provided in canvas to obtain api keys. Use ChatGPT to get the code. Make sure the file is downloaded and saved.
2.   Make sure you divide GPT code as shown in tutorials, dont make multiple requestes.


In [33]:
pip install tweepy


In [ ]:
import tweepy
import pandas as pd

# Replace with your own credentials
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAH96zQEAAAAAEV81FSKR31FV2K0%2FdG8mVNrfKk4%3DiTMs3Se7nfM1wMuayyEjhkcxxzDKVuznHOKHnIeF9bqgUrS4yF"
# Authenticate with Twitter API v2
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Define search parameters
query = "(machine learning OR artificial intelligence) -is:retweet lang:en"
tweet_count = 50  # Adjust as needed

# Fetch tweets
tweets = client.search_recent_tweets(query=query, tweet_fields=["id", "text", "author_id"], max_results=tweet_count)

# Store in a dataframe
tweet_data = [[tweet.id, tweet.author_id, tweet.text] for tweet in tweets.data]
df = pd.DataFrame(tweet_data, columns=["Tweet ID", "User ID", "Tweet Text"])

# Save to CSV
df.to_csv("tweets_data.csv", index=False)
print("✅ Tweets collected successfully and saved to 'tweets_data.csv'.")

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

Honestly, this assignment was a mix of frustrating and interesting.
This assignment was tough because of API limits and annoying SSL errors that kept messing things up. I liked working with web scraping and text processing, but the time crunch made it stressful.

What was challenging?
The biggest headache was Twitter’s API limits—getting hit with errors like 403 Forbidden and 429 Too Many Requests made it tough to collect tweets. I had to look into workarounds like snscrape or adjusting the request rate.
Scraping GitHub Marketplace was another challenge. Handling pagination to get 1000 products while making sure the script didn’t break was a bit tricky.
Cleaning the text data was fine, but NLTK issues (missing stopwords or lemmatizer errors) made it annoying at times.
What I actually enjoyed
Scraping real-world data felt rewarding. Seeing a CSV full of useful data after struggling with errors was satisfying.
Data cleaning—removing duplicates, handling missing values, and processing text—was kinda fun once everything started working smoothly.

# Write your response below
Fill out survey and provide your valuable feedback.

https://docs.google.com/forms/d/e/1FAIpQLSd_ObuA3iNoL7Az_C-2NOfHodfKCfDzHZtGRfIker6WyZqTtA/viewform?usp=dialog